# Création du graphe ADP


# Récupération des documents XML

In [1]:
# Fonctions utilitaires

def nettoyer_unicode(c):
    liste_codes =  {
        'Ã\xa0': 'à',
        'Ã€': 'À',
        'Ã¢': 'â',
        'Ã‚': 'Â',
        'Ã©': 'é',
        'Ã\x89': 'É',
        'Ã\xa8': 'è',
        'Ã\xaa': 'ê',
        'Ã\x8a': 'Ê',        
        'Ã«': 'ë',
        'Ã®':'î',
        'Ã\x8e':'Î',        
        'Ã¯': 'ï', 
        'Ã´': 'ô',
        'Ã\x94': 'Ô',
        'Ã¹': 'ù', 
        'Ã»': 'û',
        'Å\x93': 'œ', 
        'Â«': '«',
        'Â»': '»',
        'Ã§': 'ç',
        'Ã\x87': 'Ç',
        'Âº': 'º',
        'â\x80\x99': '’',
        'â\x80\xa6' : '…',
    }
    
    for code in liste_codes:
        c = c.replace(code,liste_codes[code])
        
    return c

In [2]:
import pickle

f = open("../Data/ADP/liste_adp_complete.py", 'rb') 
rawdata = pickle.load(f)

In [3]:
data = {}
for d in rawdata:
    data[d] = nettoyer_unicode(rawdata[d])

# Extraction des informations à partir des documents XML

In [4]:
import xml.etree.ElementTree as ET

productIdenfifier = {
    "01" : "Proprietary",
    "02" : "ISBN-10",
    "03" : "GTIN-13",
    "06" : "DOI",
    "13" : "LCCN" ,
    "15" : "ISBN-15",
}


productForm = {
    "00" : "Undefined",
    "BA" : "Book",
    "BB" : "Hardback",
    "BC" : "Paperback / softback",
}


primaryContentType = {
    "10" : "Text",   
}


titleElementLevel = {
    "01" : "Product",
    "02" : "Collection level",
    "03" : "Subcollection"   
}

titleDetail = {
    "00" : "Undefined",
    "01" : "Distinctive title (book); Cover title (serial); Title on item (serial content item or reviewed resource)",
    "02" : "ISSN key title of serial",
    "03" : "Title in original language",
    "04" : "Title acronym or initialism",
    "05" : "Abbreviated title",
    "06" : "Title in other language",
    "08" : "Former title", 
    "14" : "Alternative title",
}

contributorRole = {
    "A01" : "By (author)",
    "A02" : "With",
}

languageRole = {
    "01" : "Language of text",
    "02" : "Original language of a translated text",
}

subjectSchema = {
    "01" : "Dewey",
    "10" : "BISAC",
    "23" : "Publisher's own category",
    "93" : "Thema",
    "94" : "Thema place qualifier"   
}

pusblishingRole = {
    "01" : "Publisher",
    "10" : "Publishing group"
}

textType = {
    "03" : "Description",
    "06" : "Review quote",    
    "10" : "Promotional headline",
}


editeursQuebecois = [
    "Alire",
    "Atma",
    "Bayard Canada",
    "Bravo!",
    "Chouette",
    "De la Bagnole",
    "Édimag",
    "Fonfon",
    "Glénat Québec",
    "JCL",
    "La Courte échelle",
    "La Griffe",
    "La Mèche",
    "La Semaine",
    "Le Jour",
    "Les Éditions de l'Hexagone",
    "Les Éditions du Journal",
    "Les Éditions Québec-Livres",
    "Les Presses Libres",
    "Libre Expression",
    "Logiques",
    "Marcel Broquet Éditeur",
    "Marchand de feuilles",
    "Michel Quintin",
    "Monde Différent (Un)",
    "Parfum d'encre",
    "Perro éditeur",
    "Petit homme",
    "Presses Aventure",
    "Publistar",
    "Recto-Verso",
    "Soulières Éditeur",
    "Stanké",
    "Trécarré",
    "Typo",
    "VLB éditeur",
    "Wilson & Lafleur",
    "Z'Ailées",
]


In [5]:
# On retire les documents qui n'ont pas d'information sur la publication
data_nettoye = {}

print('avant:', len(data))
for item in data:
    root = ET.fromstring(data[item]) 
    publisher = root.find("./Product/PublishingDetail")
    detail = root.find("./Product/DescriptiveDetail")
    if publisher and detail:
        data_nettoye[item] = data[item]
        
print('après:', len(data_nettoye))        
    

avant: 120215
après: 120152


In [6]:
listeInfo = {}

for item in data:
    root = ET.fromstring(data[item])
    # info = root.findall('./Product/DescriptiveDetail/Audience/AudienceCodeType')
    # info = root.findall('./Product/DescriptiveDetail/Contributor/BiographicalNote')
    # info = root.findall("./Product/PublishingDetail/Publisher/PublishingRole[.='01']../PublisherName")
    # info = root.findall("./Product/PublishingDetail/CityOfPublication")
    # info = root.findall("./Product/CollateralDetail/TextContent/TextType")
    # info = root.findall('./Product/DescriptiveDetail/Collection/CollectionType')
    # info = root.findall("./Product/RelatedMaterial/RelatedProduct/ProductRelationCode")
    # info = root.findall("./Product/ProductIdentifier/ProductIDType")
    info = root.findall("./Product/PublishingDetail/PublishingDate/Date")
    # info = root.findall("./Product/DescriptiveDetail/Subject/MainSubject/../SubjectSchemeIdentifier[.='93']../SubjectCode")
     
    for i in info:
        if (i.text) in listeInfo:
            listeInfo[i.text] += 1
        else:
            listeInfo[i.text] = 1



In [ ]:
# Pour tester

for (e,n) in sorted(listeInfo.items(), key = lambda x: x[1], reverse = True):
    print(e,n)

# Description des données

## Statistiques sur les infos

* ProductIDType: tous les items ont les trois types de ID suivants: ISBN-15 (15), GTIN (03) et propriétaire (01)
* ProductForm: on trouve seulement le code BA (Book)
* ProductFormDetail: on trouve seulement le code B101 (Mass market (rack) paperback)
* PrimaryContentType: on trouve seulement le code 10 (Text)
* TitleType: on trouve seulement le code 01 (Distinctive title (book); Cover title (serial); Title on item (serial content item or reviewed resource)
* ContributionRole: on trouve seulement le code A01 (By (author))
* Contributor/SequenceNumber: On note jusqu'à 30 auteurs
    * 1 auteur:  112163
    * 2 auteurs: 18230
    * 3 auteurs: 3864
    * 4 auteurs: 983
    * 5 auteurs: 379
    * 6 auteurs: 191
    * 7 auteurs: 94
    * 8 auteurs: 55
    * 9 auteurs: 25
    * 10 auteurs et plus: 91
* Contributor/BiographicalNote: 6164 instances
* LanguageRole: on trouve seulement le code 01 (language of text)
* LanguageCode: 120 115 français, 36 anglais.
* SujbectSchemaIdentifer:
    * Thema (93): 77935
    * BISAC (10): 78954
    * Publisher's own category (23): 95818
    * Dewey (01): 860
    * Thema place qualifier (94): 300
* AudienceCodeType: on trouve seulement le code 02 (proprietary)
* CollectionType: on trouve seulement le code 10 (Publisher collection)  69305 instances trouvées
* PublishingRole: dans tous les cas on trouve les codes 01 (publisher) et 10 (publishing group)
* CityOfPublication: seulement Montréal, qui apparaît 2223 fois
* CollateralDetail/TextContent/TextType:
    * Description (03):  20049
    * Promotional headline (10): 3365
    * Review quote (06): 76
* CollateralDetail/TextContent/ContentAudience: on trouve seulement le code 00 (unrestricted)
* RelatedMaterial/RelatedProduct/ProductRelationCode:
    * Electronic version available as (27):  6478
    * Replaced by (05): 1895
    * Unspecified (00): 444
    * Is part of (02): 12
    * Alternative format (06): 8
    


## Remarques

* La liste ne contient pas que des livres édités au Québec


# Création du graphe RDF

In [7]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib import RDF, RDFS, OWL, XSD


SCHEMA = Namespace("https://schema.org/")
DCTERMS = Namespace("http://purl.org/dc/terms/")
DBO = Namespace("http://dbpedia.org/ontology/")
SOGIDES = Namespace("http://www.sogides.com/")
SOGIDES_PROP = Namespace("http://www.sogides.com/prop/")





In [8]:
grapheADP_Auteurs = Graph()
grapheADP_Auteurs.bind("schema","https://schema.org/", override=True)
grapheADP_Auteurs.bind("sog-prop","http://www.sogides.com/prop/", override=True)
grapheADP_Auteurs.bind("sog-classe","http://www.sogides.com/classe/", override=True)
grapheADP_Auteurs.bind("sog-auteur","http://www.sogides.com/auteur/", override=True)


In [9]:
grapheADP_Livres = Graph()
grapheADP_Livres.bind("schema","https://schema.org/", override=True)
grapheADP_Livres.bind("sog-prop","http://www.sogides.com/prop/", override=True)
grapheADP_Livres.bind("sog-classe","http://www.sogides.com/classe/", override=True)
grapheADP_Livres.bind("sog-livre","http://www.sogides.com/livre/", override=True)


In [10]:
grapheADP_Editeurs = Graph()
grapheADP_Editeurs.bind("schema","https://schema.org/", override=True)
grapheADP_Editeurs.bind("sog-prop","http://www.sogides.com/prop/", override=True)
grapheADP_Editeurs.bind("sog-classe","http://www.sogides.com/classe/", override=True)
grapheADP_Editeurs.bind("sog-editeur","http://www.sogides.com/editeur/", override=True)


## Création du graphe des auteurs

On ne prendra que les auteurs de livres publiés par une maisons d'édition québécoise

In [11]:
id = 0
auteurs = {}
for item in data_nettoye:
    root = ET.fromstring(data[item])
    editeur = root.findall("./Product/PublishingDetail/Publisher/PublishingRole[.='01']../PublisherName") 
    if not editeur:
        print('ERREUR',item)   

    if editeur and editeur[0].text in editeursQuebecois:
        # print(editeur[0].text)

        liste_auteurs = root.findall('./Product/DescriptiveDetail/Contributor')
        for a in liste_auteurs:               
            nom = a.find("./PersonName")
            if nom.text not in auteurs:
                id += 1
                auteurs[nom.text] = SOGIDES['auteur/' + str(id)]
                grapheADP_Auteurs.add((SOGIDES['auteur/' + str(id)],SCHEMA.name,Literal(nom.text)))
                grapheADP_Auteurs.add((SOGIDES['auteur/' + str(id)],RDF.type,SOGIDES['classe/Auteur']))
                
                bio = a.findall('./BiographicalNote')
                if bio:
                    grapheADP_Auteurs.add((SOGIDES['auteur/' + str(id)],SOGIDES_PROP.bio,Literal(bio[0].text)))

                # print(nom.text, auteurs[nom])

In [12]:
print(len(auteurs),'auteurs')
print(len(grapheADP_Auteurs), 'triplets')

5945 auteurs
13289 triplets


In [13]:
for (s,p,o) in grapheADP_Auteurs.triples((SOGIDES['auteur/335'],None,None)):
    print(s,p,o)

http://www.sogides.com/auteur/335 https://schema.org/name Roger Lemelin
http://www.sogides.com/auteur/335 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.sogides.com/classe/Auteur
http://www.sogides.com/auteur/335 http://www.sogides.com/prop/bio Roger Lemelin (1919-1992) est né à Québec dans une famille modeste de la basse-ville. Ã la suite d’un accident de ski qui le maintient au lit pendant plusieurs mois, il écrit, à 25 ans, son premier roman, <I>Au pied de la Pente douce</I>. Il a été, entre autres, correspondant canadien pour les magazines américains <I>Time </I>et <I>Life</I>, homme d’affaires, directeur du journal <I>La Presse</I>. Dans les années 1980, il a préparé avec Gilles Carle le scénario du fi lm <I>Les Plouffe </I>et celui qui fut tiré du <I>Crime d’Ovide Plouffe </I>avec Denys Arcand et Gilles Carle.


## Création du graphe des éditeurs

In [14]:
id = 0
editeurs = {}
for e in editeursQuebecois:
    id += 1
    editeurs[e] = SOGIDES['editeur/' + str(id)]
    grapheADP_Editeurs.add((SOGIDES['editeur/' + str(id)], RDF.type, SOGIDES['classe/Editeur']))   
    grapheADP_Editeurs.add((SOGIDES['editeur/' + str(id)], SCHEMA.name, Literal(e)))   

In [15]:
print(len(grapheADP_Editeurs))

76


## Création du graphe des livres

In [16]:
livres = []
for item in data_nettoye:
    root = ET.fromstring(data[item])
    editeur = root.findall("./Product/PublishingDetail/Publisher/PublishingRole[.='01']../PublisherName") 
    if not editeur:
        print('ERREUR',item)

    if editeur and editeur[0].text in editeursQuebecois:
        rf = root.find('./Product/RecordReference')
        grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], RDF.type, SOGIDES['classe/Livre']))

        grapheADP_Livres.add((SOGIDES['livre/' + rf.text], SCHEMA.publisher, editeurs[editeur[0].text]))
        liste_auteurs = root.findall('./Product/DescriptiveDetail/Contributor')
        for a in liste_auteurs:               
            nom = a.find("./PersonName")
            grapheADP_Livres.add((SOGIDES['livre/' + rf.text], SCHEMA.author, auteurs[nom.text]))

        isbn = root.findall("./Product/ProductIdentifier/ProductIDType[.='15']../IDValue")
        if isbn:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SCHEMA.isbn, Literal(isbn[0].text)))

        language = root.find('./Product/DescriptiveDetail/Language/LanguageCode')
        if language:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SCHEMA.inLanguage, Literal(language.text)))
 
        liste_thema_main_subjet = root.findall("./Product/DescriptiveDetail/Subject/MainSubject/../SubjectSchemeIdentifier[.='93']../SubjectCode")
        for t in liste_thema_main_subjet:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SOGIDES_PROP.mainSubjectThema, Literal(t.text)))
        
        liste_thema_subject = root.findall("./Product/DescriptiveDetail/Subject/SubjectSchemeIdentifier[.='93']../SubjectCode")
        for t in liste_thema_subject:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SOGIDES_PROP.subjectThema, Literal(t.text)))           
            
        titre = root.findall('./Product/DescriptiveDetail/TitleDetail/TitleElement/TitleText')
        if titre:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SCHEMA.name, Literal(titre[0].text)))
       
        description = root.findall("./Product/CollateralDetail/TextContent/TextType[.='03']../Text")
        if description:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SOGIDES_PROP.description, Literal(description[0].text)))
 
        reviewQuote = root.findall("./Product/CollateralDetail/TextContent/TextType[.='06']../Text")
        if reviewQuote:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SOGIDES_PROP.commentaire, Literal(reviewQuote[0].text)))

        promo = root.findall("./Product/CollateralDetail/TextContent/TextType[.='10']../Text")
        if promo:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SOGIDES_PROP.descriptionPromotionnelle, Literal(promo[0].text)))

            
        date = root.find("./Product/DescriptiveDetail/PublishingDate/Date")
        if promo:
            grapheADP_Livres.add((SOGIDES['livre/' + str(rf.text)], SOGIDES_PROP.descriptionPromotionnelle, Literal(promo[0].text)))
            

In [17]:
print(len(grapheADP_Livres),'triplets')

104958 triplets


In [25]:
# for (s,p,o) in grapheADP_Livres.triples((SOGIDES['livre/375883'],None,None)):
#     print(s,p,o)

for (s,p,o) in grapheADP_Livres.triples((None,SCHEMA.isbn,None)):
    print(s,p,o)


http://www.sogides.com/livre/142738 https://schema.org/isbn 9782890054509
http://www.sogides.com/livre/350240 https://schema.org/isbn 9782893819976
http://www.sogides.com/livre/143053 https://schema.org/isbn 9782890061736
http://www.sogides.com/livre/452217 https://schema.org/isbn 9782896497614
http://www.sogides.com/livre/489387 https://schema.org/isbn 9782923896120
http://www.sogides.com/livre/581159 https://schema.org/isbn 9782896074235
http://www.sogides.com/livre/384626 https://schema.org/isbn 9782894356142
http://www.sogides.com/livre/142903 https://schema.org/isbn 9780885120833
http://www.sogides.com/livre/159519 https://schema.org/isbn 9782894350300
http://www.sogides.com/livre/306741 https://schema.org/isbn 9782892496345
http://www.sogides.com/livre/401493 https://schema.org/isbn 9782923771632
http://www.sogides.com/livre/297224 https://schema.org/isbn 9782893819570
http://www.sogides.com/livre/415803 https://schema.org/isbn 9782894357231
http://www.sogides.com/livre/488783 ht

http://www.sogides.com/livre/392204 https://schema.org/isbn 9782923364506
http://www.sogides.com/livre/436568 https://schema.org/isbn 9782896497065
http://www.sogides.com/livre/455543 https://schema.org/isbn 9782897621056
http://www.sogides.com/livre/143204 https://schema.org/isbn 9782890068919
http://www.sogides.com/livre/252723 https://schema.org/isbn 9782890057548
http://www.sogides.com/livre/353262 https://schema.org/isbn 9782896601134
http://www.sogides.com/livre/347217 https://schema.org/isbn 9782896480852
http://www.sogides.com/livre/398974 https://schema.org/isbn 9782764021897
http://www.sogides.com/livre/303290 https://schema.org/isbn 9782760401136
http://www.sogides.com/livre/406093 https://schema.org/isbn 9782894356029
http://www.sogides.com/livre/458370 https://schema.org/isbn 9782897142940
http://www.sogides.com/livre/359980 https://schema.org/isbn 9782896492855
http://www.sogides.com/livre/407696 https://schema.org/isbn 9782896608577
http://www.sogides.com/livre/143492 ht

http://www.sogides.com/livre/298159 https://schema.org/isbn 9782922145458
http://www.sogides.com/livre/443694 https://schema.org/isbn 9782897033699
http://www.sogides.com/livre/303702 https://schema.org/isbn 9782764001257
http://www.sogides.com/livre/305625 https://schema.org/isbn 9782890894471
http://www.sogides.com/livre/313701 https://schema.org/isbn 9782764005682
http://www.sogides.com/livre/411138 https://schema.org/isbn 9782897032081
http://www.sogides.com/livre/306363 https://schema.org/isbn 9782891117449
http://www.sogides.com/livre/320118 https://schema.org/isbn 9782892492460
http://www.sogides.com/livre/164418 https://schema.org/isbn 9782890264274
http://www.sogides.com/livre/490474 https://schema.org/isbn 9782923708386
http://www.sogides.com/livre/359546 https://schema.org/isbn 9782764016725
http://www.sogides.com/livre/409505 https://schema.org/isbn 9782764023112
http://www.sogides.com/livre/311860 https://schema.org/isbn 9782764005385
http://www.sogides.com/livre/352570 ht

http://www.sogides.com/livre/296032 https://schema.org/isbn 9782764006344
http://www.sogides.com/livre/400846 https://schema.org/isbn 9782896494514
http://www.sogides.com/livre/346698 https://schema.org/isbn 9782923662299
http://www.sogides.com/livre/251600 https://schema.org/isbn 9782890057586
http://www.sogides.com/livre/257677 https://schema.org/isbn 9782890057920
http://www.sogides.com/livre/402240 https://schema.org/isbn 9782924259641
http://www.sogides.com/livre/367790 https://schema.org/isbn 9782890448261
http://www.sogides.com/livre/421010 https://schema.org/isbn 9782923910925
http://www.sogides.com/livre/606823 https://schema.org/isbn 9782924251652
http://www.sogides.com/livre/331896 https://schema.org/isbn 9782895683858
http://www.sogides.com/livre/306595 https://schema.org/isbn 9782892493832
http://www.sogides.com/livre/303319 https://schema.org/isbn 9782760402096
http://www.sogides.com/livre/311256 https://schema.org/isbn 9782893817675
http://www.sogides.com/livre/382512 ht

http://www.sogides.com/livre/351209 https://schema.org/isbn 9782896490998
http://www.sogides.com/livre/306275 https://schema.org/isbn 9782891116091
http://www.sogides.com/livre/422241 https://schema.org/isbn 9782894357385
http://www.sogides.com/livre/278726 https://schema.org/isbn 9782894504796
http://www.sogides.com/livre/296836 https://schema.org/isbn 9782893812854
http://www.sogides.com/livre/404470 https://schema.org/isbn 9782923910666
http://www.sogides.com/livre/281656 https://schema.org/isbn 9782894313152
http://www.sogides.com/livre/459999 https://schema.org/isbn 9782924563687
http://www.sogides.com/livre/351604 https://schema.org/isbn 9782923342115
http://www.sogides.com/livre/303514 https://schema.org/isbn 9782760405158
http://www.sogides.com/livre/411924 https://schema.org/isbn 9782764023372
http://www.sogides.com/livre/340026 https://schema.org/isbn 9782923364223
http://www.sogides.com/livre/142590 https://schema.org/isbn 9782890053946
http://www.sogides.com/livre/104146 ht

...

http://www.sogides.com/livre/303680 https://schema.org/isbn 9782764001004
http://www.sogides.com/livre/388511 https://schema.org/isbn 9782895625186
http://www.sogides.com/livre/289580 https://schema.org/isbn 9782895421771
http://www.sogides.com/livre/264046 https://schema.org/isbn 9782895420798
http://www.sogides.com/livre/420278 https://schema.org/isbn 9782764024331
http://www.sogides.com/livre/372847 https://schema.org/isbn 9782764018538
http://www.sogides.com/livre/527962 https://schema.org/isbn 9782895790259
http://www.sogides.com/livre/296754 https://schema.org/isbn 9782893813387
http://www.sogides.com/livre/414966 https://schema.org/isbn 9782764023709
http://www.sogides.com/livre/455229 https://schema.org/isbn 9782896151769
http://www.sogides.com/livre/575531 https://schema.org/isbn 9782896954391
http://www.sogides.com/livre/448558 https://schema.org/isbn 9782897220495
http://www.sogides.com/livre/316925 https://schema.org/isbn 9782764007303
http://www.sogides.com/livre/447101 ht

http://www.sogides.com/livre/416863 https://schema.org/isbn 9782923910796
http://www.sogides.com/livre/339572 https://schema.org/isbn 9782764013533
http://www.sogides.com/livre/297211 https://schema.org/isbn 9782893813967
http://www.sogides.com/livre/297647 https://schema.org/isbn 9782895680420
http://www.sogides.com/livre/298755 https://schema.org/isbn 9782892251876
http://www.sogides.com/livre/316416 https://schema.org/isbn 9782764006092
http://www.sogides.com/livre/411988 https://schema.org/isbn 9782896609833
http://www.sogides.com/livre/214829 https://schema.org/isbn 9782894311936
http://www.sogides.com/livre/320340 https://schema.org/isbn 9782980004308
http://www.sogides.com/livre/287257 https://schema.org/isbn 9782895421672
http://www.sogides.com/livre/142605 https://schema.org/isbn 0000000439497
http://www.sogides.com/livre/399115 https://schema.org/isbn 9782894356661
http://www.sogides.com/livre/404413 https://schema.org/isbn 9782764810392
http://www.sogides.com/livre/103826 ht

http://www.sogides.com/livre/382945 https://schema.org/isbn 9782897260033
http://www.sogides.com/livre/415809 https://schema.org/isbn 9782894357507
http://www.sogides.com/livre/489281 https://schema.org/isbn 9782922944204
http://www.sogides.com/livre/449883 https://schema.org/isbn 9782764026335
http://www.sogides.com/livre/266511 https://schema.org/isbn 9782895420873
http://www.sogides.com/livre/306203 https://schema.org/isbn 9782891115094
http://www.sogides.com/livre/315348 https://schema.org/isbn 9782891119979
http://www.sogides.com/livre/309284 https://schema.org/isbn 9782760406735
http://www.sogides.com/livre/464291 https://schema.org/isbn 9782897514815
http://www.sogides.com/livre/384628 https://schema.org/isbn 9782894356098
http://www.sogides.com/livre/120278 https://schema.org/isbn 0000000096553
http://www.sogides.com/livre/276219 https://schema.org/isbn 9782890067073
http://www.sogides.com/livre/296450 https://schema.org/isbn 9782891199018
http://www.sogides.com/livre/344838 ht

http://www.sogides.com/livre/403708 https://schema.org/isbn 9782764022306
http://www.sogides.com/livre/445456 https://schema.org/isbn 9782897262556
http://www.sogides.com/livre/320215 https://schema.org/isbn 9782893810928
http://www.sogides.com/livre/370113 https://schema.org/isbn 9782923715599
http://www.sogides.com/livre/170925 https://schema.org/isbn 9782921735018
http://www.sogides.com/livre/344826 https://schema.org/isbn 9782896490561
http://www.sogides.com/livre/362372 https://schema.org/isbn 9782892257342
http://www.sogides.com/livre/491533 https://schema.org/isbn 9782896957699
http://www.sogides.com/livre/528877 https://schema.org/isbn 9782895795513
http://www.sogides.com/livre/491916 https://schema.org/isbn 9782896955824
http://www.sogides.com/livre/341886 https://schema.org/isbn 9782896490653
http://www.sogides.com/livre/446027 https://schema.org/isbn 9782764026144
http://www.sogides.com/livre/320190 https://schema.org/isbn 9782893570204
http://www.sogides.com/livre/426293 ht

http://www.sogides.com/livre/316001 https://schema.org/isbn 9782764006955
http://www.sogides.com/livre/160059 https://schema.org/isbn 9782890055445
http://www.sogides.com/livre/348071 https://schema.org/isbn 9782764014264
http://www.sogides.com/livre/351352 https://schema.org/isbn 9782764014677
http://www.sogides.com/livre/385483 https://schema.org/isbn 9782923910253
http://www.sogides.com/livre/306360 https://schema.org/isbn 9782891117418
http://www.sogides.com/livre/306933 https://schema.org/isbn 9782893812373
http://www.sogides.com/livre/380989 https://schema.org/isbn 9782764808214
http://www.sogides.com/livre/466511 https://schema.org/isbn 9782897623579
http://www.sogides.com/livre/303356 https://schema.org/isbn 9782760402935
http://www.sogides.com/livre/306708 https://schema.org/isbn 9782892495881
http://www.sogides.com/livre/370922 https://schema.org/isbn 9782764018293
http://www.sogides.com/livre/297606 https://schema.org/isbn 9782764007914
http://www.sogides.com/livre/353614 ht

## Sauvegarde du graphe

In [18]:
fichier = open("../Graphes/grapheADPAuteurs.rdf","wb")
grapheADP_Auteurs.serialize(fichier) 
fichier.close()

In [19]:
fichier = open("../Graphes/grapheADPLivres.rdf","wb")
grapheADP_Livres.serialize(fichier) 
fichier.close()

In [20]:
fichier = open("../Graphes/grapheADPEditeurs.rdf","wb")
grapheADP_Editeurs.serialize(fichier) 
fichier.close()